In [1]:
import numpy as np
import math
import queue
from time import clock


In [2]:
train = np.loadtxt("pa2train.txt")
test = np.loadtxt("pa2test.txt")
validation = np.loadtxt("pa2validation.txt")

In [3]:
node_count = 0

In [65]:
class Node:
    def __init__(self):
        self.data = []
        # less than label?
        self.less_eq = None
        self.greater = None
        self.label = None
        self.decision = None
        global node_count
        node_count += 1

In [5]:
def id_three(train):
    root = Node()
    root.data = train 
    impure = []
    impure.append(root)
    
    
    while impure:
        n = impure[0]
        impure.pop(0)
        
        if pure(n):
            n.label= n.data[0][22]
        else:
            impure.extend(list(split_decision(n)))
        
    return root
    

In [66]:
def split_decision(n):
    n.less_eq = Node()
    n.greater = Node()

    d = n.data
    total = len(d)
    # tuple (category, threshold)
    most_gain = ()
    index = -1
    # essentially infinite
    ent = np.inf
    
    
    # find the best split
    for cat in range(22):
        # sort the data according category
        c = d[d[:,cat].argsort()]

        for x in range(len(c)-1):
            # data points with the same value are skipped over
            if c[x][cat] == c[x+1][cat]:
                continue
            
            thresh = (c[x][cat] + c[x+1][cat])/2
            b = ((x+1)/total)
            t_ent = (b)*entropy(c[:x+1]) + (1-b)*entropy(c[x+1:])
            
    
            if t_ent < ent:
                ent = t_ent
                most_gain = (cat, thresh)
                index = x + 1
    
    # create child nodes
    sorted_d = d[d[:,most_gain[0]].argsort()]
    n.less_eq.data = sorted_d[:index]
    n.greater.data = sorted_d[index:]
    
    # update parent
    n.decision = most_gain
    
    return n.less_eq, n.greater

In [7]:
def entropy(data):
    pos = 0
    neg = 0
    total = len(data)
    for x in data:
        if x[22] == 1:
            pos += 1
        else:
            neg += 1

    if pos == 0 or neg == 0:
        return 0
        
    ret = (pos/total)*math.log(pos/total)+(neg/total)*math.log(neg/total)
    return -ret   

In [8]:
def pure(node):
    l = node.data[0][22]
    
    for i in node.data:
        if l != i[22]:
            return False
    return True 
        

In [9]:
def error(tree, data):
    correct = 0
    for point in data:
        if point[22] == traverse(tree, point):
            correct += 1
    return 1 - (correct/len(data))
            

In [10]:
def traverse(node, point):
    while node.label is None:
        if point[node.decision[0]] < node.decision[1]:
            node = node.less_eq
        else: 
            node = node.greater
    return node.label
            

In [81]:
def prune(tree, validation):
    q = queue.Queue()
    q.put((tree, validation))
    
    while not q.empty():
        
        node, data = q.get()
        
        majority, error_majority = major(data)
        
        if error_majority < error(node, data):

            node.label = majority
            node.decision = None
            break
            
        if node.label is None:
            # split data
            less_eq_data = [point for point in data if point[node.decision[0]] <= node.decision[1]]
            greater_data = [point for point in data if point[node.decision[0]] > node.decision[1]]

            
            q.put((node.less_eq, less_eq_data))
            q.put((node.greater, greater_data))
            
        

In [82]:
def major(data):
    counter = 0 
    majority = 0
    
    for point in data:
        counter += point[22]
    if counter > (len(data)/2):
        majority = 1
    else:
        counter = len(data) - counter

    return majority, 1-(counter/len(data))
    

In [83]:
timer = clock() 
node_count =  0
tree = id_three(train)
print(clock() - timer)

38.29915712494676


In [84]:
train_err = error(tree, train)
print(train_err)

0.0


In [85]:
test_err = error(tree, test)
print(test_err)

0.17300000000000004


In [86]:
# One iteration of pruning
prune(tree, validation)
valid_err = error(tree, validation)
print(valid_err)
test_err = error(tree, test)
print(test_err)

0.122
0.11699999999999999


In [87]:
# 2nd iteration
prune(tree, validation)
valid_err = error(tree, validation)
print(valid_err)
test_err = error(tree, test)
print(test_err)

0.10699999999999998
0.10299999999999998


1. 
                                                (Feature 5, Threshold = 0.5) 
                                                        2000 Points
                                           /                                     \
                  (Feature 1, Threshold = 415000)                             (Feature 5, Threshold = 1.5)
                            1319 Points                                                681 Points
                  /                         \                                        /                      \
 (Feature 17, Threshold = 2506.5)(Feature 21, Threshold = 208)   (Feature 20, Threshold = 584.5) (Feature 21, Threshold = 2006)
               1284 Points                  35 Points                         292 Points                  389 Points
                            
                            
2. The training error was 0 and the test error was 0.173

3. First iteration the validation error was 0.122 and test error was 0.117.
   The second ietration the validation error was 0.107 and the test error was 0.103
   
4. The first split was based on the payment delay in September so that is probably the most prominent feature that predicts        defaults.